In [ ]:
knitr::opts_chunk$set(echo = TRUE)

Queries:
- Given that the reflectance 
- Is the means of deciding wv sensitivity range optimal?

In [ ]:
library('readr')
library('dplyr')
library('ggplot2')
library('tibble')

TriOS units are in mW/(m^2 nm Sr), so convert to W later.

In [ ]:
df2 <- read_delim('spec_data_curated.txt',delim='\t')

The dataframe is summarized is follows:

In [ ]:
summary(df2)

Get the bin width of wavelengths in nm.

In [ ]:
binwidth <- round(sum(diff(sort(unique(df2$wv))))/length(diff(sort(unique(df2$wv)))),2)
print(paste('The bin width is',binwidth,'nm'))

We can ignore the rep and fit a spline to data.(Need to divide by the number of
reps at that wv, material.) We can also normalize to the highest value, for each
material. We test in the range 425 to 575.

In [ ]:
df2 %>% 
  filter(between(wv,425,575)) %>% group_by(material,white) %>%
  summarize(tote_quanta = sum(quanta*binwidth)/n_distinct(rep),
              hi_quanta = (median(quanta)*binwidth)/n_distinct(rep)    ) %>%
  group_by(material) %>% mutate(norm_quanta = tote_quanta/max(tote_quanta)
                                )-> photon_n

In [ ]:
summary(photon_n)

The reflectance (in the range relevant for urchins, measured in photons) for the
darkest material is 3.7% of that for the brightest material. The reflectanc of the
darkest third of ink values are extremely similar. 

This plateau is a combination of (i) an actual tapering of the change in
refletance and (ii) some measurement error - although the replicates are
mostly very similar.

I want to find the ink shades which best correspond to specified reflectances. I
will treat the darkest shade available to me as 0: no reflectance. There will
be less specular reflectance and scattered light in the water - I will measure
this with the patterns. 

I will re-normalize the curve having subtracted the reflectance at white=0
(the darkest ink, dark = 100), and bringing negative quanta values to 0. 


In [ ]:
photon_n %>%
  group_by(material) %>%
    mutate(abs_reflectance =  tote_quanta / (max(tote_quanta)),
           reflectance = (tote_quanta - min(tote_quanta)) / (max(tote_quanta) - min(tote_quanta)),
        dark = 100*(-(white - 0.5) + 0.5)     
        ) -> photon_n

head(photon_n)

The differences for the darkest shades, recognisable by (human) eye, are minute. We use a relative scale of how white the inks are (from 0 to 1) as this is postively related to reflectance, and therefore intuitive.

The loess must be adjusted such that the trend increases monotonically. With
span=0.3 this is achieved.

In [ ]:
photon_n %>% 
  ggplot(aes(x=white,y=abs_reflectance,color=material,shape=material)) + 
  labs(plot.title ='Number of photons in the range 425 to 575 nm') +
  stat_smooth(method="loess",formula='y~x',span=0.3,level=0.95) + 
  geom_point() + theme_classic()

In [ ]:
fit = loess(data=photon_n[(photon_n$material=="Thick"),],
            formula='abs_reflectance~white',span=0.3,degree=2)
round(predict(fit,newdata=c(0,0.05,0.1,0.2,0.33,0.5,0.66,1)),5)

This number of quanta is negligibly small in contrast with the maximum - all
reflectances this low (which will be asigned NA) can be assigned to the darkest
(most ink) shade value. Similarily, maximum reflectance values will be rounded 
up to the lightest shade. 

# Invert the model equation

We want to get the ideal shade from a known reflectance. Here, I revert to using the dark ink shade. We use a relative measure of reflectance,normalized such that the minimum reflectance is zero (rel_reflectance). Here with fit polynomials:

In [ ]:
photon_n %>% 
  filter(material=="Thick") %>%
  ggplot(aes(x=reflectance,y=dark)) + 
  stat_smooth(method="lm",formula='y~I(x**(0.3))',  color='green',level=.999) + 
  geom_hline(aes(yintercept=0)) +
  geom_point() + theme_classic()

This is quite close - and does not one falls below zero *dark* (ink) - but doesn't maximize the black ink (I want the dark part to be as black as possible). 

In [ ]:
photon_n %>% 
  filter(material=="Thick") %>%
  ggplot(aes(x=reflectance,y=dark)) + 
  stat_smooth(method="loess",formula='y~x',span=0.3,level=0.95) + 
  geom_hline(aes(yintercept=0)) +
  geom_point() + theme_classic()

This loess fit finds the minimum and maximum but the non-monotonic kink is concerning I had a go at using a basic spline but the lm above is still the best fit I used. 

#### Fit a model and save the obj

In [ ]:
photon_n %>% filter(material=="Thick") -> Thick_photon_n
lm.root.3.fit   <- lm(dark~I(reflectance**(.3)),data=Thick_photon_n)

Plot predictions of ink against reflectance.

In [ ]:
newdat <- as.data.frame(seq(from=0,to=1,length.out = 1000))
names(newdat) <- as.character("reflectance")
predz         <- round(predict(lm.root.3.fit,newdata=newdat),2)
plot(NULL, xlim=c(0,1),ylim=c(0,100),xlab="Reflectance",ylab="ink")
lines(x=newdat$reflectance,y=predz)
saveRDS(lm.root.3.fit,"reflectance.rds")

In [ ]:
summary(predz)

Above are the reflectances (proportion) of the material sought and the ink values (%)
of the maximum ink value, to be used. So, for example, to get 50% of the maximal
reflectance (the median), an ink proportion of 17% of the maximum ought to be used. 

In [ ]:
fogra39_richblack <- c(91, 79, 62, 97)
print(paste(c('C:','M:','Y:','K:'),round(fogra39_richblack*round(predict(lm.root.3.fit,
      newdata=0.5 # reflectance sought
      ),2)/100,2)))

## Test robustness of estimates

The reflectance indicated is subject to (i) range of values over which it is
computed, (ii) the shape of that function, a square wave in this case, (iii) 
the distribution of wavelengths of light applied. 

*Check if should use norm.quanta instead*

In [ ]:
count_quanta <- function(df,lambda_max,range){
hi <- lambda_max + range/2
lo <- lambda_max - range/2
  
df %>% 
  filter(between(wv,lo,hi)) %>% group_by(material,white) %>%
  summarize(tote_quanta = sum(quanta*binwidth)/n_distinct(rep)) %>%
  group_by(material) %>% mutate(norm_quanta = tote_quanta/max(tote_quanta)) %>%
  group_by(material) %>%
  mutate(
  reflectance = (tote_quanta - min(tote_quanta)) /
                (max(tote_quanta) - min(tote_quanta)),
         dark = 100*(-(white - 0.5) + 0.5) ) -> photon_n

invertfit = loess(data=photon_n[(photon_n$material=="Thick"),],
            formula='dark ~ reflectance',span=0.3,degree=2)
reflect <- seq(from=0,to=1,length.out = 21)
dark         <- round(predict(invertfit,newdata=reflect),2)
table <- rbind(reflect,dark)
return(table)
}